# Setup

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from scipy import optimize
import statistics as stat
from IPython.display import display

class par: None
class moms: None
    
# from farhi and gourio
class true: None
true.beta = 0.961
true.mu = 1.079
true.delta = 0.02778
true.alpha = 0.244
true.g_Z = 0.01298
true.r_star = np.nan
true.g_T = np.nan
true.p = 0.034

## Settings and assumptions

In [2]:
# a. shock size
par.b = -np.log(1-0.15)

# b. risk aversion coefficient
par.theta = 12

# c. IES, sigma = 1/IES
par.sigma = 0.5

In [3]:
# Tries whether this file is called from main or internally
# Ensures that we can run this code on it's own
try: 
    called_from_main
except: 
    country = "US"
    start = 1984
    end = 2000
    called_from_main = 0 # if 0, print results in this notebook but don't store

# Load data

In [4]:
if country == "UK": 
    df = pd.read_csv("data/V3_timeseries__postrev_I1Y_19-Oct-2018-uk.csv", sep=";")
else: 
    df = pd.read_csv("data/V3_timeseries__postrev_I1Y_19-Oct-2018.csv")
    
df = df[['year','ProfitK','ProfitY','RF', 'growthpop','priceinvt','PD','growthTFP','ik','EmpPop']]
df = df.set_index('year')
#display(df.head())
#display(df.describe())

# Moments

In [5]:
# Start and end dates defined in main notebook
moms.ProfitK = stat.mean(df.loc[start:end,"ProfitK"])/100
moms.s_K = stat.mean(df.loc[start:end,"ProfitY"])/100
moms.ik = stat.mean(df.loc[start:end,"ik"])/100
moms.PD = stat.mean(df.loc[start:end,"PD"])
moms.TFPgrowth = stat.mean(df.loc[start:end,"growthTFP"])/100
moms.RF = stat.mean(df.loc[start:end,"RF"])/100 #used in step 3

#for name in ['ProfitK','s_K','ik','PD','TFPgrowth','RF']:
#    print(f'{name:10s} = {getattr(moms,name):.5f}')

# Step 1

Estimate $g_L$, $g_Q$ and $\bar{N}$ directly from data

In [6]:
par.g_L = stat.mean(df.loc[start:end,"growthpop"])/100
par.g_Q = -stat.mean(df.loc[start:end,"priceinvt"])/100 # note: negativ value used
par.N_bar = stat.mean(df.loc[start:end,"EmpPop"])/100

#for name in ['g_L','g_Q','N_bar']:
#    print(f'{name:5s} = {getattr(par,name):.5f}')

# Step 2

Estimate $\mu$, $\alpha$, $\delta$, $g_z$ by solving equation system

In [7]:
def eq_footnote_15(par,moms):
     return moms.TFPgrowth - (par.g_T-(1-moms.s_K)*par.g_L-moms.s_K*(par.g_T+par.g_Q))

def eq_11(par,moms):
    return (1+par.g_T) - (1+par.g_L)*(1+par.g_Z)**(1/(1-par.alpha))*(1+par.g_Q)**(par.alpha/(1-par.alpha))

def eq_15(par,moms):

    qstar = 1 # normalization
    beta_star = 1/(1+par.r_star)
    
    temp = (1+par.g_Q)/beta_star-(1-par.delta)
    kstar = (temp*par.mu/(par.alpha*qstar))**(1/(par.alpha-1))*par.N_bar
    ystar = kstar**par.alpha*par.N_bar**(1-par.alpha)
    pistar = (par.mu+par.alpha-1)/par.mu*ystar

    return moms.ProfitK-pistar/(kstar/qstar) 
    
def eq_18(par,moms):
    return (1+par.g_Q)*(1+par.g_T)-(1-par.delta) - moms.ik

def eq_20(par,moms):
    return moms.s_K - (par.mu+par.alpha-1)/(par.mu)

def eq_23(par,moms):
    beta_star = 1/(1+par.r_star)
    return moms.PD - beta_star*(1+par.g_T)/(1-beta_star*(1+par.g_T))

#def eq_27(par,moms):
#    return moms.ProfitK - (par.mu+par.alpha-1)/par.alpha*(par.r_star+par.delta+par.g_Q)

In [8]:
def set_parameters(par,x):
    
    for name,value in zip(par.names,x):
        setattr(par,name,value)
    
def set_x(par):
    
    x = np.zeros(len(par.names))
    for i,name in enumerate(par.names):
        x[i] = getattr(par,name)
        
    return x

def eq_system(x,par,moms):
    
    # a. set parameters
    set_parameters(par,x)            
    
    # c. evaluate equations
    out = []
    out.append(eq_footnote_15(par,moms))
    out.append(eq_11(par,moms))
    out.append(eq_15(par,moms))          
    out.append(eq_18(par,moms))
    out.append(eq_20(par,moms))
    out.append(eq_23(par,moms))

    return out

In [9]:
# a. parameters to estimate
par.names = ['g_Z','g_T','delta','alpha','r_star','mu']

# b. guess
par.mu = 1.01
par.delta = 0.025
par.alpha = 0.25
par.g_Z = 0.08 #0.02
par.r_star = 0.05
par.g_T = 0.04
x = set_x(par)

# c. solve
solution = optimize.fsolve(eq_system, x, args=(par,moms), full_output=0)
set_parameters(par,solution)

# d. print result
#for name in par.names:
#    print(f'{name:10} = {getattr(par,name):.8f} [FH: {getattr(true,name):.4f}]')        

# Step 3

Estimate $\beta$ and $p$

In [10]:
def update_misc(par):
    
    par.beta_star = 1/(1+par.r_star)
    par.bh = -np.log((2-np.exp(-par.b)))
    par.g_PC = (1+par.g_T)/(1+par.g_L)-1
    

In [11]:
def find_p(p,par,moms):
        
    update_misc(par)        
    MOM2 = ((1-2*p)+p*np.exp(-par.bh*(1-par.theta)) + p*np.exp(-par.b*(1-par.theta)))
    MOM3 = ((1-2*p)+p*np.exp(-par.bh*(-par.theta)) + p*np.exp(-par.b*(-par.theta))) 
    
    return moms.RF - (MOM2/(par.beta_star*MOM3)-1)

In [12]:
par.p = optimize.fsolve(find_p, 0.02, args=(par,moms), full_output=0)[0]
update_misc(par)

In [13]:
MOM =  ((1-2*par.p)+par.p*np.exp(-par.bh*(1-par.theta)) + par.p*np.exp(-par.b*(1-par.theta)))**((1-par.sigma)/(1-par.theta)) 
par.beta = par.beta_star/((1+par.g_PC)**(-par.sigma)*MOM);

# Store or print values

In [14]:
# Collect and store results if called from main notebook
if called_from_main == 1:

    df_estimates = []
    for name in ['beta','mu','p','delta','alpha','g_L', 'g_Z', 'g_Q', 'N_bar','r_star']:
        df_estimates.append({'Name': name, str(start) + ' - ' + str(end):getattr(par,name)})
            
    df_estimates = pd.DataFrame(df_estimates)
    df_estimates = df_estimates.set_index('Name')

    # Magic - stores across notebooks
    %store df_estimates
    
# Print results if not called from main notebook
if called_from_main == 0: 
    for name in ['beta','mu','p','delta','alpha','g_L', 'g_Z', 'g_Q', 'N_bar','r_star']:
        print(f'{name:10} = {getattr(par,name):4f}')
    del called_from_main

beta       = 0.960540
mu         = 1.078717
p          = 0.034146
delta      = 0.027777
alpha      = 0.243684
g_L        = 0.011706
g_Z        = 0.012984
g_Q        = 0.017692
N_bar      = 0.623442
r_star     = 0.059389
